In [1]:
import pandas as pd
import numpy as np

In [66]:
plays = pd.read_csv('data/lastfm/user_artists.dat', sep='\t')
artists = pd.read_csv('data/lastfm/artists.dat', sep='\t', usecols=['id','name'])

# Merge artist and user pref data
ap = pd.merge(artists, plays, how="inner", left_on="id", right_on="artistID")
ap = ap.rename(columns={"weight": "playCount"})

# Group artist by name
artist_rank = ap.groupby(['name']) \
    .agg({'userID' : 'count', 'playCount' : 'sum'}) \
    .rename(columns={"userID" : 'totalUsers', "playCount" : "totalPlays"}) \
    .sort_values(['totalPlays'], ascending=False)

artist_rank['avgPlays'] = artist_rank['totalPlays'] / artist_rank['totalUsers']
print(artist_rank)

                    totalUsers  totalPlays     avgPlays
name                                                   
Britney Spears             522     2393140  4584.559387
Depeche Mode               282     1301308  4614.567376
Lady Gaga                  611     1291387  2113.563011
Christina Aguilera         407     1058405  2600.503686
Paramore                   399      963449  2414.659148
...                        ...         ...          ...
Morris                       1           1     1.000000
Eddie Kendricks              1           1     1.000000
Excess Pressure              1           1     1.000000
My Mine                      1           1     1.000000
A.M. Architect               1           1     1.000000

[17632 rows x 3 columns]


In [67]:
# Merge into ap matrix
ap = ap.join(artist_rank, on="name", how="inner") \
    .sort_values(['playCount'], ascending=False)

# Preprocessing
pc = ap.playCount
play_count_scaled = (pc - pc.min()) / (pc.max() - pc.min())
ap = ap.assign(playCountScaled=play_count_scaled)
#print(ap)

# Build a user-artist rating matrix 
ratings_df = ap.pivot(index='userID', columns='artistID', values='playCountScaled')
n_users, n_items = ratings_df.shape
print("rating matrix shape", ratings_df.shape)

ratings = ratings_df.fillna(0).values
# Show sparsity
sparsity = float(len(ratings.nonzero()[0])) / (ratings.shape[0] * ratings.shape[1]) * 100
print("sparsity: %.2f" % sparsity)

rating matrix shape (1892, 17632)
sparsity: 0.28


In [68]:
from scipy.sparse import csr_matrix

# Build a sparse matrix
X = csr_matrix(ratings)
print(ratings_df)

artistID  1      2         3      4      5         6      7      8      9      \
userID                                                                          
2           NaN    NaN       NaN    NaN    NaN       NaN    NaN    NaN    NaN   
3           NaN    NaN       NaN    NaN    NaN       NaN    NaN    NaN    NaN   
4           NaN    NaN       NaN    NaN    NaN       NaN    NaN    NaN    NaN   
5           NaN    NaN       NaN    NaN    NaN       NaN    NaN    NaN    NaN   
6           NaN    NaN       NaN    NaN    NaN       NaN    NaN    NaN    NaN   
...         ...    ...       ...    ...    ...       ...    ...    ...    ...   
2095        NaN    NaN       NaN    NaN    NaN       NaN    NaN    NaN    NaN   
2096        NaN    NaN       NaN    NaN    NaN       NaN    NaN    NaN    NaN   
2097        NaN    NaN       NaN    NaN    NaN       NaN    NaN    NaN    NaN   
2099        NaN    NaN       NaN    NaN    NaN       NaN    NaN    NaN    NaN   
2100        NaN    NaN  0.00

In [63]:
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm.data import Dataset

# Build data references + train test
Xcoo = X.tocoo()
data = Dataset()
data.fit(np.arange(n_users), np.arange(n_items))
interactions, weights = data.build_interactions(zip(Xcoo.row, Xcoo.col, Xcoo.data)) 
train, test = random_train_test_split(interactions)

# Train
model = LightFM(learning_rate=0.05, loss='warp')
model.fit(train, epochs=10, num_threads=2)

In [64]:
# Predict
scores = model.predict(0, np.arange(n_items))

top_items = ratings['item_labels'][np.argsort(-scores)]

array([-1.5528026 , -0.53155154, -3.3965557 , ..., -2.3380501 ,
       -2.065186  , -2.508964  ], dtype=float32)